In [3]:
import boto3
import pandas
from sqlalchemy import create_engine
import psycopg2

In [4]:
# importing credentials from credentials file
from credentials import (
    AccessKey,
    SecretKey,
    BucketName,
    DatabaseHost,
    UserName,
    Password,
    TableName,
    DatabaseName
)

In [5]:
s3r = boto3.resource('s3', aws_access_key_id=AccessKey, aws_secret_access_key=SecretKey)
bucket = s3r.Bucket(BucketName)
file_list = ['bengals.csv', 'boyd_receiving.csv', 'chase_receiving.csv', 'higgins_receiving.csv']
# make list of file names to remove redundant code to download files from S3 bucket
for file in file_list:
    bucket.download_file(file, file)


In [6]:
bengals_data = pandas.read_csv('./bengals.csv')
boyd_data = pandas.read_csv('./boyd_receiving.csv')
chase_data = pandas.read_csv('./chase_receiving.csv')
higgins_data = pandas.read_csv('./higgins_receiving.csv')

In [7]:
# Renaming Indivudal Player's Stats Columns for Joining/Readability Purposes
boyd_data = boyd_data.rename(columns={"Yards": "Boyd Yards", "TD": "Boyd TD"})
chase_data = chase_data.rename(columns={"Yards": "Chase Yards", "TD": "Chase TD"})
higgins_data = higgins_data.rename(columns={"Yards": "Higgins Yards", "TD": "Higgins TD"})

In [8]:
total_data = bengals_data.merge(boyd_data, how='left').merge(chase_data, how='left').merge(higgins_data, how='left')
# perform left joins starting with Bengals since Bengals will contain all Weeks including Bye weeks, regardless of
# individual player injuries

In [9]:
# Since individual players preseason stats are not given, i.e. Null after merge, we first want to take care of
# filtering out the bye week since the Location, Opponent, and Result fields are Null for this column.
# Then we can take care of replacing all Null values in the table with zeroes because all that will be left are players
# individual stats for Preseason Games, or games they did not play. Meaning, it is safe to assume we can assign these
# Null values to zero. We must first handle the bye weeks before doing this since having '0' for Location and Opponent
# Does not make sense. More importantly, assigning the 'Result' for the Bye week to '0' will indicate a Loss in the upcoming
# steps - which is inaccurate. 

# first we want to filter out the Bye week - for the Bye week - Location, Opponent, and Result are Null
total_data = total_data.dropna(subset=['Location', 'Opponent', 'Result'])

# Now the only Nulls that should exist are stats for games in which the individual players did not participate
# We can safely assign these values to zero
total_data = total_data.fillna(0)


In [10]:
total_data['Result'] = total_data['Result'].map({1.0: "Win", 0.0: "Loss"})

In [11]:
connection_str = f"postgresql://{UserName}:{Password}@{DatabaseHost}/{DatabaseName}"
engine = create_engine(connection_str)
total_data.to_sql(TableName, engine, if_exists='replace')

24